# Веб-парсинг данных с помощью Python, Jupyter, BeautifulSoup и Pandas (на примере таблицы игроков NBA)

## Введение

Начиная проект анализа данных исследователь всегда ставит перед собой и заказчиком ряд важных вопросов.
Первая группа вопросов - это вопросы типа "зачем". Задача этих вопросов - корректно оценить объем проекта (контекст, потребности, видение, возможные результаты), позволяюшего превратить "сырые" данные в знания, используя ваши аналитические навыки и умения.

Следующая группа вопросов содержит вопросы типа "как". Здесь происходит разработка решения - вы собираете данные, чтобы использовать методы и алгоритмы статистики и машинного обучения чтобы раскрыть эти идеи, реализуя ваши технические навыки. Именно этот раздел мы будем обсуждать в данном блокноте.

Содержание блокнота:
- Считывание и очистка табличных данных с веб-сайта (на примере данных NBA из Basketball Reference).
- Сохранение наших данных в csv - создание собственного набора данных.

## Необходимые действия по настройке среды Python

Установим (если они ранее не были установлены) библиотеки BeautifulSoup4, requests и pandas - инструменты, которые мы будем использовать для очистки и анализа наших данных:

> pip install beautifulsoup4

> pip install requests

> pip install pandas

## Настройка Jupyter Notebook
Теперь, когда у нас установлены наша виртуальная среда и пакеты, мы можем начать с открытия Jupyter Notebooks, где мы будем кодировать.

Для работв с проектом рекомендуется создать отдельную папку (например, **scraping_nba_data**), а в ней - подпапку для собранных данных (например, **web_scraping**).

Затем нам нужно перейти в этот новый каталог (web_scraping) в нашем терминале / командной строке, например:

cd Desktop/scraping_nba_data/web_scraping/

Файл ноутбука следует поместить в эту папку, после чего, находясь в папке **web_scraping** открыть Jupyter Notebook, введя это в командной строке:

> jupyter notebook

В результате выполнения этой команды автоматически откроется ваш браузер, где вы увидите «Список записной книжки».

Теперь можно открыть блокнот и приступить к работе.

## Импорт библиотек и модулей
Вот список библиотек и модулей, который мы планируем использовать:

- requests (для считывания HTML)
- BeautifulSoup (для разбора данных)
- pandas (для управления таблицами данных)
- os (для управления каталогами при сохранении данных)

Вот как это должно выглядеть:

In [ ]:
import requests # для считывания HTML
from bs4 import BeautifulSoup # для разбора данных
import pandas as pd # для управления таблицами данных
import os # для управления каталогами при сохранении данных

## Определим, какие данные нам необходимы для очистки
Важно с самого начала определить цель парсинга. Мы не хотим очищать данные, которые нам на самом деле не нужны.
В этом блокноте мы разберем извлечение данных из Справочника по баскетболу https://www.basketball-reference.com/, в частности таблицу результативности Игроков по играм https://www.basketball-reference.com/leagues/NBA_2020_per_game.html , которые представляют собой статистику игровой эффективности игроков для ~ 500 игроков НБА. Страница баскетбольного справочника выглядит так:
![image.png](attachment:image.png)

Вот данные, которые мы получим от каждого игрока в сезоне 2019–2020 годов:

- **Player**: Сезонная игра
- **Pos**: Должность
- **Age**: возраст игрока на 1 февраля сезона.
- **Tm**: Команда
- **G**: Сыгранные игры
- **GS**: Игры начаты
- **MP**: Минуты в игре
- **FG**: Полевые цели
- **FGA**: Попытки забить
- **FG%**: процент забитых мячей
- **3P**: Трехочковые броски с игры
- **3PA**: Трехочковые попытки с игры
- **3P%**: процент попаданий с игры с 3-х очков
- **2P**: 2-х очковые попытки забить
- **2P%**: Процент 2-х очковых попаданий
- **eFG%**: Эффективное процентное соотношение полевых целей
- **FT**: Свободные броски
- **FTA**: Попытки свободного броска
- **FT%**: Процент свободного броска
- **ORB**: Подборы в атаке
- **DRB**: Подборы в защите
- **TRB**: Всего подборов
- **AST**: голевые передачи
- **STL**: Отъемы
- **BLK**: Блоки
- **TOV**: Обороты за игру
- **PF**: Персональные фолы
- **PTS**: Очки

## Рассмотрим URL-адрес
Для начала давайте посмотрим на URL страницы, которую мы хотим очистить.
Вот что мы видим в URL:

> https://www.basketball-reference.com/leagues/NBA_2020_per_game.html

Cохраним этот URL, чтобы наш парсер знал, какую страницу мы очищаем.

## Запрос содержимого URL

In [ ]:
# Декомпозируем доступ к URL:
# переменная, которую мы создаем и которой назначаем URL.
url = 'https://c.1440.space/pages/viewpage.action?pageId=19840134'
# переменная, которую мы создаем для хранения нашего request.get действия.
page = requests.get(url)
# метод, который мы используем для получения содержимого URL
print(page)

ConnectionError: HTTPSConnectionPool(host='c.1440.space', port=443): Max retries exceeded with url: /pages/viewpage.action?pageId=19840134 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7ee01b5a39d0>: Failed to resolve 'c.1440.space' ([Errno -2] Name or service not known)"))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Очистка данных с использованием BeautifulSoup
Когда мы выводим page, как в последней строке нашего кода выше, мы видим <Response [200]>. Чтобы увидеть содержимое страницы - page, нам нужно обратиться к полю page.content:

In [ ]:
page.content

b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/build" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202402151" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://cmp.quantcast.com\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \n\t\t    \'/choice.js?tag_version=V2\');\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insertBefore(element, firstSc

В таком виде данные выглядят несколько неаккуратно. Чтобы упростить чтение, мы можем передать содержимое нашей неупорядоченной страницы через BeautifulSoup:

In [ ]:
soup=BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

Output hidden; open in https://colab.research.google.com to view.

Декомпозируем применение BeautifulSoup:
- **soup** - это переменная, которую мы создаем для назначения метода BeatifulSoup, который задает желаемый формат результатов с помощью анализатора HTML. Это позволяет Python читать компоненты страницы, а не рассматривать ее как одну длинную строку.
- **print(soup.prettify())** напечатает то, что мы получили в более структурированном и читаемом древовидном формате


## Извлечение HTML
Наш следующий шаг - извлечь HTML-содержимое нашей таблицы и заголовка.
Мы делаем это, проверяя HTML (щелкнув правой кнопкой мыши страницу, которую мы очищаем, и выберем «Проверить элемент»).
Отсюда мы видим, что каждая строка каждого игрока описывается HTML-классом **full_table**:
![image.png](attachment:image.png)
Давайте сохраним это содержимое таблицы в переменной **table** с помощью команды **soup.find_all**. Здесь вы можете видеть, что мы сохранили все строки типа **full_table** в нашу переменную **table**.


In [ ]:
table = soup.find_all(class_='full_table')
table

[<tr class="full_table"><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Adams,Steven" data-append-csv="adamsst01" data-stat="player"><a href="/players/a/adamsst01.html">Steven Adams</a></td><td class="center" data-stat="pos">C</td><td class="right" data-stat="age">26</td><td class="left" data-stat="team_id"><a href="/teams/OKC/2020.html">OKC</a></td><td class="right" data-stat="g">63</td><td class="right" data-stat="gs">63</td><td class="right" data-stat="mp_per_g">26.7</td><td class="right" data-stat="fg_per_g">4.5</td><td class="right" data-stat="fga_per_g">7.6</td><td class="right" data-stat="fg_pct">.592</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right non_qual" data-stat="fg3_pct">.333</td><td class="right" data-stat="fg2_per_g">4.5</td><td class="right" data-stat="fg2a_per_g">7.5</td><td class="right" data-stat="fg2_pct">.594</td><td class="right" data-stat="efg_pct">.5

Теперь нам нужно сохранить заголовки столбцов. Проделав то же самое, что и выше, мы видим, что заголовок столбца имеет класс thead:
![image.png](attachment:image.png)

Сохраним наш заголовок в переменной **head**:

In [ ]:
head = soup.find(class_='thead')
column_names_raw=[head.text for item in head][0]
column_names_raw

'\nRk\nPlayer\nPos\nAge\nTm\nG\nGS\nMP\nFG\nFGA\nFG%\n3P\n3PA\n3P%\n2P\n2PA\n2P%\neFG%\nFT\nFTA\nFT%\nORB\nDRB\nTRB\nAST\nSTL\nBLK\nTOV\nPF\nPTS\n'

Вы можете видеть выше, что наши данные имеют \n между каждым фрагментом данных, для дальнейшей работы с данными их необходимо очистить:

In [ ]:
column_names_clean=column_names_raw.replace("\n",",",).split(",")[2:-1]
column_names_clean

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']

Декомпозируем действия по извлечению и очистке HTML:
- **head** - это переменная, которую мы будем использовать для хранения всех заголовков столбцов.
- **soup.find** извлекает все div контейнеры с атрибутом class thead.
- **column_names_raw** - это переменная, которую мы будем использовать для хранения необработанных данных, которые мы извлекаем и которые еще не очищены.
- **[head.text for item in head][0]** выбирает первую строку в переменной head.
- **column_names_clean** - это переменная, которую мы будем использовать для хранения окончательно очищенных данных.
- **column_names_raw.replace("\n",",").split(",")[2:-1]** заменяет \n запятой.
- **.split(",")** разделяет заголовок каждого столбца на отдельные фрагменты данных в кавычках, как указано выше.

## Цикл разбора данных
Теперь мы можем в цикле перебрать все элементы **table** и для каждой строки данных (в каждом HTML-теге **td** для каждого игрока) мы можем извлечь всю статистику игрока и сохранить ее в список под названием **players**.

In [ ]:
players=[]
for i in range(len(table)):
    player_=[]
    for td in table[i].find_all("td"):
        player_.append(td.text)
    players.append(player_)
df=pd.DataFrame(players, columns=column_names_clean).set_index("Player")
# Очистим имена игроков от случайных спецсимволов
df.index=df.index.str.replace('*','')

<ipython-input-8-baf5b1b67568>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.index=df.index.str.replace('*','')


Декомпозируем действия в цикле:
- **players** - список, в котором мы храним данные, которые собираем.
- **for** цикл используется для перебора последовательности. Наша последовательность - это каждый **td** контейнер div, который мы сохранили в **table**.
- **player_** список, который будет содержать все данные, когда мы перебираем каждый **td**.
- **for td in table[i].find_all("td")** - это вложенный цикл for, который будет перебирать все контейнеры **td**.
- **player.append(td.text)** перемещает данные **TD** в наш список **player_**.
- **players.append(player_)** заносит все данные, сохраненные в **player_**, в наш окончательный список **players**.
- **df** - это имя нашего датафрейма pandas.
- **pd.DataFrame(players, columns = column_names_clean).set_index("Player")** настраивает индексное поле датафрейма.
- **df.index = df.index.str.replace('*', '')** очищает имя игрока от специальных символов.

## Наш набор данных
Наконец, мы можем посмотреть, как выглядит наш набор данных:

In [ ]:
print(df.shape)
df.head(5)

(529, 28)


,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
Steven Adams,C,26,OKC,63,63,26.7,4.5,7.6,.592,0.0,...,.582,3.3,6.0,9.3,2.3,0.8,1.1,1.5,1.9,10.9
Bam Adebayo,PF,22,MIA,72,72,33.6,6.1,11.0,.557,0.0,...,.691,2.4,7.8,10.2,5.1,1.1,1.3,2.8,2.5,15.9
LaMarcus Aldridge,C,34,SAS,53,53,33.1,7.4,15.0,.493,1.2,...,.827,1.9,5.5,7.4,2.4,0.7,1.6,1.4,2.4,18.9
Kyle Alexander,C,23,MIA,2,0,6.5,0.5,1.0,.500,0.0,...,,1.0,0.5,1.5,0.0,0.0,0.0,0.5,0.5,1.0
Nickeil Alexander-Walker,SG,21,NOP,47,1,12.6,2.1,5.7,.368,1.0,...,.676,0.2,1.6,1.8,1.9,0.4,0.2,1.1,1.2,5.7


У нас 529 строк с 28 столбцами данных - теперь мы можем сохранить наши данные в файл csv:
    
# Запись данных в файл csv:

In [ ]:
df.to_csv('nba_data_2020.csv', header=True)

- **df.to_csv** - метод, который мы используем, который перемещает наш фрейм данных в файл csv.
- **nba_data_2020.csv** - имя CSV-файла.
- **header=True** - обеспечивает сохранение заголовка в файле csv.

## Заключение
Итак, мы настроили среду Python, проанализировали HTML-документ, извлекли данные из table и организовали их в DataFrame, чтобы создать наш собственный набор данных в CSV!
## Дополнительные материалы
Работа с текстовым парсингом описана в статье:

> https://zdrons.ru/veb-programmirovanie/parsing-sajtov-na-python-v-jupyter-notebook-legkij-sposob-dlya-poiska-kljuchevyh-fraz/

Работа с парсингом слабоструктурированных данных описана в статье:
> https://idatica.com/blog/parsing-saytov-na-python-rukovodstvo-dlya-novichkov/